# **Mask RCNN**

## **Importing Modules**

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from mrcnn import visualize
from mrcnn.model import log
from mrcnn.config import Config
from mrcnn import model as modellib, utils
from IPython.display import clear_output


In [1]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Mask R-CNN and utility imports
from mrcnn import visualize
from mrcnn.model import log
from mrcnn.config import Config
from mrcnn import model as modellib, utils
from IPython.display import clear_output

# Import your custom dataset class
from CustomDataset import CustomDataset


## **Loading dataset & Model configuration**

In [2]:
# Root directory of the project
ROOT_DIR = os.getcwd()
DATASET_DIR = os.path.join(ROOT_DIR,"Dataset")
print("DATASET_DIR:",DATASET_DIR)


sys.path.append(ROOT_DIR)

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR,"mask_rcnn_coco.h5")
if not os.path.exists(COCO_WEIGHTS_PATH):
    utils.download_trained_weights(COCO_WEIGHTS_PATH)

# Directory to save logs and model checkpoints
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR,"logs")
print("DEFAULT_LOGS_DIR:",DEFAULT_LOGS_DIR)

DATASET_DIR: /Users/fatimanevrekar/Desktop/Plant_Health_Detection/Plant_Health_MaskRCNN/Dataset
DEFAULT_LOGS_DIR: /Users/fatimanevrekar/Desktop/Plant_Health_Detection/Plant_Health_MaskRCNN/logs


In [ ]:
import h5py

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
print("COCO_WEIGHTS_PATH:", COCO_WEIGHTS_PATH)

# Check if the COCO weights file exists, if not download it
if not os.path.exists(COCO_WEIGHTS_PATH):
    print("COCO weights not found, downloading...")
    utils.download_trained_weights(COCO_WEIGHTS_PATH)
else:
    print("COCO weights found.")

# Now check if the file is indeed an HDF5 file
try:
    with h5py.File(COCO_WEIGHTS_PATH, 'r') as file:
        print(f"Successfully opened {COCO_WEIGHTS_PATH} with h5py.")
except IOError as e:
    print(f"Cannot open {COCO_WEIGHTS_PATH} with h5py. Error: {e}")
    print("Attempting to re-download the COCO weights.")
    os.remove(COCO_WEIGHTS_PATH)  # Remove the corrupt file
    utils.download_trained_weights(COCO_WEIGHTS_PATH)


In [3]:
from mrcnn.config import Config

class CustomConfig(Config):
    """Configuration for training on your custom dataset.
    Derives from the base Config class and overrides values specific to your dataset.
    """
    # Give the configuration a recognizable name
    NAME = "custom_object_detection"

    # Number of classes (including background)
    NUM_CLASSES = 1 + 6  # Background + your number of classes

    # Number of training and validation steps per epoch
    STEPS_PER_EPOCH = 100
    VALIDATION_STEPS = 50

    # Other configurations can be modified according to your requirements like
    # GPU_COUNT, IMAGES_PER_GPU, LEARNING_RATE, etc.


In [ ]:
class CustomConfig(Config):
    NAME = "object"

    # Processing no of images based on GPU configuration
    IMAGES_PER_GPU = 4

    # Number of classes (including background)
    NUM_CLASSES = 1 + 6

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 102
    
    # Number of validation steps per epoch
    VALIDATION_STEPS = 25
    
    USE_MINI_MASK = False
    
    TRAIN_ROIS_PER_IMAGE = 128
    print("TRAIN_ROIS_PER_IMAGE:",TRAIN_ROIS_PER_IMAGE)

    
    MASK_SHAPE = [28, 28]
    
class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):
        print("dataset_dir:",dataset_dir)

        # Adding classes
        self.add_class("object", 1, "Potato_Early_Blight")
        self.add_class("object", 2, "Potato_Healthy")
        self.add_class("object", 3, "Potato_Late_Blight")
        self.add_class("object", 4, "Tomato_Healthy")
        self.add_class("object", 5, "Tomato_Leaf_Mold")
        self.add_class("object", 6, "Tomato_Leaf_Spot")

        # Setting up dataset path
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)
        print("dataset_dir:",dataset_dir)


        # Loading JSON file
        annotations1 = json.load(open(os.path.join(dataset_dir, 'annotations.json')))
        print("annotations1:",annotations1)

        annotations = list(annotations1.values())
        print("annotations1:",annotations)


        # Getting images with annotations
        annotations = [a for a in annotations if a['regions']]

        # Getting x and y coordinates of polygon (regions)
        for a in annotations:
            polygons = [r['shape_attributes'] for r in a['regions']]
            objects = [s['region_attributes']['names'] for s in a['regions']]
            print("objects:",objects)
            name_dict = {"Potato_Early_Blight": 1,"Potato_Healthy": 2,"Potato_Late_Blight": 3,
                         "Tomato_Healthy": 4,"Tomato_Leaf_Mold": 5,"Tomato_Leaf_Spot": 6}

            num_ids = [name_dict[a] for a in objects]

            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",
                image_id=a['filename'],
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    # Generating instance masks for an image
    def load_mask(self, image_id):

        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Getting indexes of pixels inside the polygon and set them to 1
        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [ ]:
# Root directory of the project
ROOT_DIR = os.getcwd()
DATASET_DIR = os.path.join(ROOT_DIR, "Dataset")
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

In [7]:
# Configuration for the model
config = CustomConfig()

In [8]:
# Create the model in training mode
model = modellib.MaskRCNN(mode="training", config=config, model_dir=DEFAULT_LOGS_DIR)


In [11]:
# Load weights trained on MS COCO, but exclude the last layers because
# the number of classes is different from COCO's 80 classes
model.load_weights(COCO_WEIGHTS_PATH, by_name=True, exclude=[
    "mrcnn_class_logits", "mrcnn_bbox_fc", 
    "mrcnn_bbox", "mrcnn_mask"])
clear_output()


In [13]:
# Load training and validation datasets
dataset_train = CustomDataset()
dataset_train.load_custom(DATASET_DIR, "train")
dataset_train.prepare()

dataset_val = CustomDataset()
dataset_val.load_custom(DATASET_DIR, "val")
dataset_val.prepare()

dataset_dir: /Users/fatimanevrekar/Desktop/Plant_Health_Detection/Plant_Health_MaskRCNN/Dataset
dataset_dir: /Users/fatimanevrekar/Desktop/Plant_Health_Detection/Plant_Health_MaskRCNN/Dataset


In [ ]:
# Train the model
# Note: You might want to adjust the learning rate and epochs based on your dataset
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=2,
            layers='all')

Generating layer regex ...
Generating train data ...
Generating val data ...
Creating callbacks ...
Adding custom callbacks ...
Setting params ...

Starting at epoch 0. LR=0.001

Checkpoint Path: /Users/fatimanevrekar/Desktop/Plant_Health_Detection/Plant_Health_MaskRCNN/logs/custom_object_detection20231118T1307/mask_rcnn_custom_object_detection_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_bra

/Users/fatimanevrekar/anaconda3/envs/fan7402venv/lib/python3.8/site-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2023-11-18 13:08:44.157479: W tensorflow/c/c_api.cc:304] Operation '{name:'res4s_branch2a/bias/Assign' id:4127 op device:{requested: '', assigned: ''} def:{{{node res4s_branch2a/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](res4s_branch2a/bias, res4s_branch2a/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-11-18 13:08:46.644630: W tensorflow/c/c_api.cc:304] Operation '{name:'range' id:19033 op device:{requested: '', assigned: ''} def:{{{node range}} = Range[Tidx=DT_INT32, _has_manual_control_depen

2023-11-18 13:08:50.647884: W tensorflow/c/c_api.cc:304] Operation '{name:'truediv_13' id:20574 op device:{requested: '', assigned: ''} def:{{{node truediv_13}} = RealDiv[T=DT_FLOAT, _has_manual_control_dependencies=true](mul_18, Cast_13)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-11-18 13:08:50.807439: W tensorflow/c/c_api.cc:304] Operation '{name:'truediv_14' id:20582 op device:{requested: '', assigned: ''} def:{{{node truediv_14}} = RealDiv[T=DT_FLOAT, _has_manual_control_dependencies=true](mul_19, Cast_14)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-11-18 13:08:50.967798: W tensorflow/c/c_api.cc:304] Operation '{name:'truediv_15' id:20590 

2023-11-18 13:08:53.892391: W tensorflow/c/c_api.cc:304] Operation '{name:'truediv_33' id:20734 op device:{requested: '', assigned: ''} def:{{{node truediv_33}} = RealDiv[T=DT_FLOAT, _has_manual_control_dependencies=true](mul_38, Cast_33)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-11-18 13:08:54.057284: W tensorflow/c/c_api.cc:304] Operation '{name:'truediv_34' id:20742 op device:{requested: '', assigned: ''} def:{{{node truediv_34}} = RealDiv[T=DT_FLOAT, _has_manual_control_dependencies=true](mul_39, Cast_34)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-11-18 13:08:54.217759: W tensorflow/c/c_api.cc:304] Operation '{name:'truediv_35' id:20750 

2023-11-18 13:08:57.202387: W tensorflow/c/c_api.cc:304] Operation '{name:'truediv_53' id:20894 op device:{requested: '', assigned: ''} def:{{{node truediv_53}} = RealDiv[T=DT_FLOAT, _has_manual_control_dependencies=true](mul_58, Cast_53)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-11-18 13:08:57.371653: W tensorflow/c/c_api.cc:304] Operation '{name:'truediv_54' id:20902 op device:{requested: '', assigned: ''} def:{{{node truediv_54}} = RealDiv[T=DT_FLOAT, _has_manual_control_dependencies=true](mul_59, Cast_54)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-11-18 13:08:57.535430: W tensorflow/c/c_api.cc:304] Operation '{name:'truediv_55' id:20910 

...

2023-11-18 13:09:32.365821: W tensorflow/c/c_api.cc:304] Operation '{name:'add_metric_2/total/Assign' id:22813 op device:{requested: '', assigned: ''} def:{{{node add_metric_2/total/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](add_metric_2/total, add_metric_2/total/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-11-18 13:09:32.573189: W tensorflow/c/c_api.cc:304] Operation '{name:'range_8' id:22839 op device:{requested: '', assigned: ''} def:{{{node range_8}} = Range[Tidx=DT_INT32, _has_manual_control_dependencies=true](range_8/start, Rank_8, range_8/delta)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them

Training ...


2023-11-18 13:09:34.049408: W tensorflow/c/c_api.cc:304] Operation '{name:'add_metric_4/count/Assign' id:22894 op device:{requested: '', assigned: ''} def:{{{node add_metric_4/count/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](add_metric_4/count, add_metric_4/count/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Epoch 1/2


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/fatimanevrekar/anaconda3/envs/fan7402venv/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/fatimanevrekar/anaconda3/envs/fan7402venv/lib/python3.8/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'CustomConfig' on <module '__main__' (built-in)>


## **Training Model**

In [ ]:
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config, model_dir=DEFAULT_LOGS_DIR)

# Load COCO weights or load the last model you trained
# Skip the loading of certain layers due to different number of classes
# Here we are excluding the layers that are different between the COCO dataset and your new dataset
model.load_weights(COCO_WEIGHTS_PATH, by_name=True, exclude=[
    "mrcnn_class_logits", "mrcnn_bbox_fc", 
    "mrcnn_bbox", "mrcnn_mask"])

# Training dataset
dataset_train = CustomDataset()
dataset_train.load_custom(DATASET_DIR, "train")
dataset_train.prepare()

# Validation dataset
dataset_val = CustomDataset()
dataset_val.load_custom(DATASET_DIR, "val")
dataset_val.prepare()

# Train the model
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=2,
            layers='all')


In [ ]:
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config, model_dir=DEFAULT_LOGS_DIR)

model.load_weights(COCO_WEIGHTS_PATH, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

# Training dataset.
dataset_train = CustomDataset()

dataset_train.load_custom(DATASET_DIR,"train")
dataset_train.prepare()

# Validation dataset
dataset_val = CustomDataset()
dataset_val.load_custom(DATASET_DIR, "val")
dataset_val.prepare()
clear_output()

In [ ]:
%%time
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=2,
            layers='all')

In [ ]:
import time

start_time = time.time()

model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=5,
            layers='all')

end_time = time.time()
print("Training took {:.2f} seconds".format(end_time - start_time))


In [ ]:
%%time
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=10,
            layers='all')

In [ ]:
%%time
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=15,
            layers='all')

In [ ]:
%%time
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=20,
            layers='all')

In [ ]:
%%time
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=25,
            layers='all')

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

## **Testing Model**

In [ ]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from mrcnn import visualize
from mrcnn.model import log
from mrcnn.config import Config
from mrcnn import model as modellib, utils
from IPython.display import clear_output 

In [ ]:
from numpy import expand_dims
from numpy import mean
from mrcnn.utils import compute_ap, compute_recall
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image

In [ ]:
ROOT_DIR = os.getcwd()

MODEL_DIR = os.path.join(ROOT_DIR, "logs")

CUSTOM_DIR = os.path.join(ROOT_DIR, "Dataset")

In [ ]:
class CustomConfig(Config):
    NAME = "object"

    # Processing no of images based on GPU configuration
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 6
    
    USE_MINI_MASK = False
    
    MASK_SHAPE = [56, 56]

In [ ]:
class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):

        # Adding classes
        self.add_class("object", 1, "Potato_Early_Blight")
        self.add_class("object", 2, "Potato_Healthy")
        self.add_class("object", 3, "Potato_Late_Blight")
        self.add_class("object", 4, "Tomato_Healthy")
        self.add_class("object", 5, "Tomato_Leaf_Mold")
        self.add_class("object", 6, "Tomato_Leaf_Spot")

        # Setting up dataset path
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # Loading JSON file
        annotations1 = json.load(open(os.path.join(dataset_dir, 'annotations.json')))
        annotations = list(annotations1.values())

        # Getting images with annotations
        annotations = [a for a in annotations if a['regions']]

        # Getting x and y coordinates of polygon (regions)
        for a in annotations:
            polygons = [r['shape_attributes'] for r in a['regions']]
            objects = [s['region_attributes']['names'] for s in a['regions']]
            print("objects:",objects)
            name_dict = {"Potato_Early_Blight": 1,"Potato_Healthy": 2,"Potato_Late_Blight": 3,
                         "Tomato_Healthy": 4,"Tomato_Leaf_Mold": 5,"Tomato_Leaf_Spot": 6}

            num_ids = [name_dict[a] for a in objects]

            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",
                image_id=a['filename'],
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    # Generating instance masks for an image
    def load_mask(self, image_id):

        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Getting indexes of pixels inside the polygon and set them to 1
        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [ ]:
dataset = CustomDataset()
dataset.load_custom(CUSTOM_DIR, "val")
dataset.prepare()
clear_output()

In [ ]:
# Loading model
config = CustomConfig()
model_ = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

weights_path = os.path.join(MODEL_DIR, "object20230110T1509/mask_rcnn_object_0025.h5")
model_.load_weights(weights_path, by_name=True)
clear_output()

In [ ]:
def get_ax(rows=1, cols=1, size=12):
  _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
  return ax

In [ ]:
# Display results
def detect_hlth(pth):
    image1 = mpimg.imread(pth)
    results1 = model_.detect([image1], verbose=1)
    ax = get_ax(1)
    r1 = results1[0]
    lbl_list=["Potato_Early_Blight","Potato_Healthy","Potato_Late_Blight","Tomato_Healthy","Tomato_Leaf_Mold",
              "Tomato_Leaf_Spot","Detection Error: Unable to detect, Try again with valid image",""]
    
    empty_list=["","","","","","",""]
    try:
        cls_value=r1['class_ids'][0]
    except IndexError:
        cls_value=7

    if cls_value==1:
        c1=lbl_list[0]
        c2,c3,c4,c5,c6,err=lbl_list[7],lbl_list[7],lbl_list[7],lbl_list[7],lbl_list[7],lbl_list[7]
    elif cls_value==2:
        c2=lbl_list[1]
        c1,c3,c4,c5,c6,err=lbl_list[7],lbl_list[7],lbl_list[7],lbl_list[7],lbl_list[7],lbl_list[7]
    elif cls_value==3:
        c3=lbl_list[2]
        c1,c2,c4,c5,c6,err=lbl_list[7],lbl_list[7],lbl_list[7],lbl_list[7],lbl_list[7],lbl_list[7]
    elif cls_value==4:
        c4=lbl_list[3]
        c1,c2,c3,c5,c6,err=lbl_list[7],lbl_list[7],lbl_list[7],lbl_list[7],lbl_list[7],lbl_list[7]
    elif cls_value==5:
        c5=lbl_list[4]
        c1,c2,c3,c4,c6,err=lbl_list[7],lbl_list[7],lbl_list[7],lbl_list[7],lbl_list[7],lbl_list[7]
    elif cls_value==6:
        c6=lbl_list[5]
        c1,c2,c3,c4,c5,err=lbl_list[7],lbl_list[7],lbl_list[7],lbl_list[7],lbl_list[7],lbl_list[7]
    else:
        err=lbl_list[6]
        c1,c2,c3,c4,c5,c6=lbl_list[7],lbl_list[7],lbl_list[7],lbl_list[7],lbl_list[7],lbl_list[7]
    
    visualize.display_instances(image1, r1['rois'], r1['masks'], r1['class_ids'],empty_list,r1['scores'], 
                                ax=ax,title=c1 or c2 or c3 or c4 or c5 or c6  or err)

In [ ]:
pth = input("Enter potato or tomato leaf image:")
pth2 = pth.replace('"','')
detect_hlth(pth2)

## **Evaluation**

In [ ]:
# Evaluation
APs = list()
ARs = list()
for image_id in dataset.image_ids:
    image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, config, image_id, use_mini_mask=False)
    scaled_image = mold_image(image, config)
    sample = expand_dims(scaled_image, 0)
    yhat = model_.detect(sample, verbose=0)
    r = yhat[0]
    AP, precisions, recalls, overlaps = compute_ap(gt_bbox, gt_class_id, gt_mask, 
                                                   r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    AR, positive_ids = compute_recall(r["rois"], gt_bbox)
    ARs.append(AR)
    
mAP = mean(APs)
mAR = mean(ARs)

In [ ]:
print("Accuracy (mAP):",round(mAP,3))

In [ ]:
#F1-Score
F1_score_= (2 * mAP * mAR)/(mAP + mAR)
print("F1 Score:",round(F1_score,3))